In [ ]:
import requests
from dotenv import load_dotenv
import os 
load_dotenv(".env")
RAPID_API_KEY = os.getenv("RAPID_API_KEY")


In [16]:
from haystack.components.routers import TransformersTextRouter

text_router = TransformersTextRouter(model="cardiffnlp/twitter-roberta-base-sentiment")
text_router.warm_up()

In [ ]:
from haystack import component, Document
from typing import Any, Dict, List, Union

@component
class YelpReviewSentimentExtractor:
    @component.output_types(documents=List[Dict])
    def run(self, url: str, headers: Dict, querystring: Dict) -> List[Document]:
        """Extracts reviews from Yelp API and performs sentiment analysis on the reviews.
        
        Args:
            url (str): Yelp API URL.
            headers (Dict): Request headers.
            querystring (Dict): Request query parameters.
        
        Returns:
            List[Document]: List of documents with sentiment analysis results."""
        reviews = self._extract_reviews(url, headers, querystring)
        reviews_document = self._sentiment_analysis(reviews)
        
        return {"documents" : reviews_document}
    
    def _extract_reviews(self, url: str, headers: Dict, querystring: Dict) -> List[Dict]:
        """Extracts reviews from Yelp API.
        
        Args:
            url (str): Yelp API URL.
            headers (Dict): Request headers.
            querystring (Dict): Request query parameters.
            
        Returns:
            List[Dict]: List of reviews.
        """
        try:
            response = requests.get(url, headers=headers, params=querystring)

            results = response.json()
            
            reviews = [{"text":results['reviews'][i]['text'],
                        "rating": results['reviews'][i]['rating'],
                        "url": results['reviews'][i]['url']} \
                        for i in range(len(results['reviews']))]
            
            return reviews
        except Exception as e:
            return []
            print(e)
            
    def _sentiment_analysis(self, reviews: List[Dict]) -> List[Document]:
        """Performs sentiment analysis on the reviews.

        Args:
            reviews (List[Dict]): List of reviews.

        Returns:
            List[Document]: List of documents with sentiment analysis results.
        """
        sent_results = []

        for idx, query in enumerate(reviews):
            result = text_router.run(text=query['text'])
            document = Document(id=idx, 
                                content=query['text'],
                                meta= {"rating": query['rating'],
                                       "url": query['url'],
                                    "sentiment": {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}.get(next(iter(result)), "Unknown")}  # Mapping the label
                                )

            sent_results.append(document)
        return  sent_results



In [74]:
url = "https://yelp-business-reviews.p.rapidapi.com/reviews/RJNAeNA-209sctUO0dmwuA"
querystring = {"sortBy":"lowestRated"}

headers = {
	"x-rapidapi-key": RAPID_API_KEY,
	"x-rapidapi-host": "yelp-business-reviews.p.rapidapi.com"
}

extract_reviews = YelpReviewSentimentExtractor()
extract_reviews.run(url=url, headers=headers, querystring=querystring)

/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/venvs/dev/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


{'documents': [Document(id=0d1168dec6c0eb79445dce8ec2122cef5ca72e59b73dd33613c5a45e3dca5f8f, content: 'We went midday around 2:30 in the afternoon on a Tuesday. Got drinks at the bar to wait for a table....', meta: {'rating': 1, 'url': 'https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=DK8kbUwWK2eui1zpV7weDw', 'sentiment': 'positive'}),
  Document(id=1, content: 'Service was okay, but probably the nastiest food I think I've ever had. If you like good food do not...', meta: {'rating': 1, 'url': 'https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=XP59MFlpQjk9R0TmGZJvuA', 'sentiment': 'negative'}),
  Document(id=2, content: 'They block people who have negative comments towards their food on facebook. That in itself is enoug...', meta: {'rating': 1, 'url': 'https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=GSTi6NUjBeyylXrF13uz5w', 'sentiment': 'negative'}),
  Document(id=3, content: 'Ordered medium burger; it was well done. Fries were cold. Cheese curds were over cooked. Service wa

In [75]:
querystring = {"sortBy":"highestRated"}
extract_reviews.run(url=url, headers=headers, querystring=querystring)

{'documents': [Document(id=2c8ffde6555f5115d0e49c3b82c225753b5d120b94bdb4ba92732b3e1bb0d20b, content: 'Loved the Perch Fish Fry! And the brand Old Fashioneds. Good Wisconsin Comfort Food!
  Will return nex...', meta: {'rating': 5, 'url': 'https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=naVdpxs7sGfpS64YbGH2fA', 'sentiment': 'positive'}),
  Document(id=1, content: 'Welcoming staff, very efficient in service and neat menu selection. Food was tasty. They really move...', meta: {'rating': 5, 'url': 'https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=zx6c92BAen8L1P-fm-xX0Q', 'sentiment': 'positive'}),
  Document(id=2, content: 'The old fashion is delicious!! The food came out in about 10-15 mins after getting our drink so spee...', meta: {'rating': 5, 'url': 'https://www.yelp.com/biz/RJNAeNA-209sctUO0dmwuA?hrid=qay0pQZFWaov_iO3CMd0Bg', 'sentiment': 'positive'}),
  Document(id=3, content: 'I'd give them 4.5 stars but bumped it up due to the host and our server being spot on.   Stopped 